In [121]:
# to start the playwright session:
# 
# # PC
## "C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9223 --user-data-dir="C:\my-chrome-profile"

# mac
# /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9223 --user-data-dir="~/chrome-debug-profile"


###################################

# to start the server which will communicate with the playwright session:
## C:/Users/dusan/OneDrive/Desktop/pp/test/crawl/.venv/Scripts/python.exe fastAPIServ.py

Running on Windows


In [2]:
# ignore the following:
## source crawl/bin/activate
#pip install google-generativeai Pillow opencv-python pytesseract playwright

In [1]:

%load_ext autoreload
%autoreload 2

import helper as hlp
import get_2fa as g2

import hlogger as hlog

In [8]:


# --- Configuration ---
site_wide_instructions = "Quickbooks Online Website"
workflow_id = "QB_batch" # unique identifier for the workflow
workflow_instructions = """Inactivate several customers in a batch action. On the customer page (found through the sales page), you should be able to select several customers and take batch actions like inactivate""" #instructions for the workflow
extra_instructions = ""
logger = hlog.HierarchicalLogger( json_file_path=workflow_id+'/logger.json')
logger.set_workflow(workflow_id)
logger.set_workflow_value("site_wide_instructions", site_wide_instructions)
logger.set_workflow_value("workflow_instructions", workflow_instructions)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# todo: 
# test what happens when a rerun is needed
# add "generalized instructions" to the actions

import time
import os
import json
import copy


wd = os.getcwd()  # Gets the current working directory
max_run = 7
max_rerun = 4
max_run_rerun = 4

max_chunks = 2
overlap_percentage = 20
draw_dots = True

browser_start_width = 1252
browser_start_height = 1252

# --- Main Loop ---
workflow_memory = hlp.reset_workflow(workflow_id, workflow_instructions, extra_instructions)

# Initialize tracking data
# track_data = {
#     "workflow_id": workflow_id,
#     "workflow_instructions": workflow_instructions,
#     "runs": []
# }


run_id = 0
rerun_ct = 0
run_rerun_ct = 0
advice = ''
summary = ''
goal_achieved = False
any_actions_succeeded = False
resize_data = None  # Store resize data between runs

# Main loop

actions_succeeded = False
page_changed = False
overall_goal_success = False


# 1) Set context
sample_id = logger.set_sample()
logger.set_rerun(rerun_ct)
logger.set_run(run_id)
logger.set_run_retry(run_rerun_ct)


# dusan_tracker['goal']=workflow_instructions

resize_browser_result = hlp.resize_browser_window(width=browser_start_width, height=browser_start_height)
run_rerun_path = ''


while run_id <= max_run and rerun_ct <= max_rerun and not overall_goal_success and run_rerun_ct <= max_run_rerun:

    
    print(logger.context)
    # deep copy run_rerun_path
    previous_run_rerun_path = copy.deepcopy(run_rerun_path)
    print(f"Now previous = {previous_run_rerun_path} which is run {run_rerun_path}")
    run_rerun_path = os.path.join(wd, workflow_id,str(sample_id),str(rerun_ct),str(run_id),str(run_rerun_ct))
    print(f"And Run is now: {run_rerun_path}")

    # create a folder for the run if it doesn't exist, if exists do nothing
    

    # dusan_tracker.append("RUN ITERATION START: run id: " + str(run_id) + " rerun count: " + str(rerun_ct) + " goal achieved: " + str(goal_achieved)+ " can continue: " + str(can_continue)+ " advice succeeded: " + str(advice))

    # dusan_tracker['run']={run_id:{'rerun':{rerun_ct:{}}}}

    # dusan_tracker['run'][run_id]['rerun'][rerun_ct]={'page_changed':page_changed,'actions_succeeded':actions_succeeded,'app_actions':[]}



    #run_data = hlp.initialize_run_data(run_id, rerun_ct)
    start_time = time.time()

    # Set up folders
    
    if run_id == 0:
        # First run in a rerun sequence OR stalled - get fresh screenshot and metadata
        # NOTICE: this needs to be done only once per rerun sequence (unless the browser is messed with durnin the run)
        
        url_metadata, screenshot_data = hlp.run_metadata_gather(
            output_path=run_rerun_path+"/chunks",
            overlap_percentage=overlap_percentage,
            max_chunks=max_chunks
        )

        if rerun_ct == 0:
            initial_state_url_metadata = url_metadata

        # WARNING, if we go beyond the first chunk, we will have to redo these hardcoding of the file names
# FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\dusan\\OneDrive\\Desktop\\pp\\test\\crawl\\v12\\amazon_screw2\\1\\0\\0\\0chunks\\resized_chunk_1.png'

        resize_screenshot_result_all_chunks = {}
        chunk_file_list =  hlp.get_files_with_extension(run_rerun_path+"/chunks", ".png")
        for f in chunk_file_list:
            tmp_resize_screenshot_result = hlp.resize_and_crop(
                input_path=f'{run_rerun_path}/chunks/{f}',
                output_path=f'{run_rerun_path}/chunks/resized_{f}',
            )
            resize_screenshot_result_all_chunks[f] = tmp_resize_screenshot_result

        resize_screenshot_result = resize_screenshot_result_all_chunks['chunk_1.png']

        # Store resize data for future runs
        resize_data = {
            'resize_browser_result': resize_browser_result,
            'resize_screenshot_result': resize_screenshot_result,
            'url_metadata': url_metadata,
            'screenshot_data': screenshot_data
        }

        # if resize_screenshot_result['original_width']==resize_screenshot_result['new_width'] and resize_screenshot_result['original_height']==resize_screenshot_result['new_height']:
        #     #dusan_tracker['run'][run_id]['rerun'][rerun_ct]['app_actions'].append('no resizing occured')
        # else:
        #     #dusan_tracker['run'][run_id]['rerun'][rerun_ct]['app_actions'].append('Resizing was necessary')

        # TODO: not sure why we have this. We can remove the _orig version up top and stick to resized.png
        # hlp.copy_and_rename_file(
        #     f'{workflow_step_path}/resized_orig.png',
        #     f'{workflow_step_path}/resized.png'
        # )
        
        # # Add step to run data for first run screenshot and analysis data 
        # hlp.add_step_to_run_data(run_data, 'initial_analysis', 'Initial screenshot and analysis', {
        #     "url_metadata": url_metadata,
        #     "screenshot_data": screenshot_data,
        #     "resize_screenshot_result": resize_screenshot_result
        # })

    else:
        # Subsequent runs - copy screenshot from temp and reuse resize data


        try: # we write to temp so that we can compare once we have the new screenshot

            # temp was captured for eval in the 0th and all other runs

            print(f"Copying screenshot from temp to new run folder: {previous_run_rerun_path}/temp to {run_rerun_path}/chunks")
            if len(previous_run_rerun_path) == 0:
                print("Error!!!!!!!!!!!!!!!! copying screenshot to new run folder: previous_run_rerun_path is empty")
            temp_file_list =  hlp.get_files_with_extension(previous_run_rerun_path+"/temp", ".png")
            for tf in temp_file_list:
                hlp.copy_and_rename_file(
                    f'{previous_run_rerun_path}/temp/{tf}',
                    f'{run_rerun_path}/chunks/{tf}',
                    delete_original=False
                )

            # Reuse resize data from first run
            resize_browser_result = resize_data['resize_browser_result']
            resize_screenshot_result = resize_data['resize_screenshot_result']
            url_metadata = resize_data['url_metadata']
            screenshot_data = resize_data['screenshot_data']
            
            # hlp.add_step_to_run_data(run_data, 'reused_data', 'Using copied data from previous run', {
            #     "url_metadata": url_metadata,
            #     "screenshot_data": screenshot_data,
            #     "resize_screenshot_result": resize_screenshot_result
            # })
            
        except Exception as e:
            print(f"Error!!!!!!!!!!!!!!!! copying screenshot to new run folder: {e}")
            continue

    page_action_json_ranked = hlp.perform_llm_analysis(
            workflow_memory,
            extra_instructions,
            site_wide_instructions,
            run_rerun_path,
            workflow_instructions,
            resize_screenshot_result,
            run_id,
            start_time
            ,advice
            ,summary
        )

    if page_action_json_ranked is None:
        print("Error!!!!!!!!!!!!!!!!!!!: LLM analysis failed to produce valid results")
        continue
    #hlp.add_step_to_run_data(run_data, 'llm_analysis', 'LLM analysis for action planning', page_action_json_ranked)

    # print("\nDrawing elements and creating highlighted screenshot...")
    # print(f"Page action JSON: {json.dumps(page_action_json_ranked, indent=2)}")


    logger.set_run_retry_value("summary_of_steps_so_far", page_action_json_ranked['summary_of_steps_so_far'])
    logger.set_run_retry_value("expected_outcome_hopeful", page_action_json_ranked['expected_outcome_hopeful'])
    logger.set_run_retry_value("_advice_assessment", page_action_json_ranked['_advice_assessment'])

    # We are only using the draw_elements function to draw the elements, and taking the screenshot is now done after actions have been done.
    hlp.draw_elements(
        page_action_json_ranked,
        draw_dots=draw_dots,
        diameter=20,
        introduce_wait = False  # Add this
    )

    # dusan: chaning


    import requests
    screenshot_response = requests.get(
        f"http://127.0.0.1:8000/screenshot",
        params={"output_path": run_rerun_path+"/dots", "overlap_percentage": overlap_percentage,"max_chunks": max_chunks, 
                    "action_id": None,
                    "candidate_id": None
                }
    )


    dot_file_list =  hlp.get_files_with_extension(run_rerun_path+"/dots", ".png")
    for df in dot_file_list:
        hlp.resize_and_crop(
            f'{run_rerun_path}/dots/{df}',
            f'{run_rerun_path}/dots/resized_{df}'
        )
## left off

    # Prepare actions to perform
    actions_to_perform = []
    for task in page_action_json_ranked.get('action_tasks', []):
        logger.set_action(task['action_id'])
        logger.set_action_value("description", task['description'])
        # Fix the string concatenation by ensuring proper order of operations
        # dusan_tracker.append("    -Action tasks: ID:" + str(task['action_id']) + " Descript: " + str(task['description']))

        for c in task['candidates']:
            logger.set_candidate(c['candidate_id'])
            logger.set_candidate_value(key="element_description", value=c['element_description'])
            logger.set_candidate_value(key="type_text", value=c.get('type_text', ''))
            logger.set_candidate_value(key="keyboard_action", value=c.get('keyboard_action', ''))
            logger.set_candidate_value(key="action", value=c['action'])
            logger.set_candidate_value(key="to_act", value=c['to_act'])

        action_candidates = {
            'description': task['description'],
            'action_id': task['action_id'],
            'candidates': [c for c in task['candidates'] if c.get('to_act', False)]
        }



        actions_to_perform.append(action_candidates)




    # Prepare the action request payload
    action_request_payload = {
        'multiple_steps_required': page_action_json_ranked.get('multiple_steps_required', False),
        'visible_elements_from_instructions': page_action_json_ranked.get('visible_elements_from_instructions', ''),
        'summary_of_steps_so_far': page_action_json_ranked.get('summary_of_steps_so_far', ''),
        'action_tasks': actions_to_perform,
        'error': page_action_json_ranked.get('error', None),
        'page_description': page_action_json_ranked.get('page_description', ''),
        'expected_outcome_hopeful': page_action_json_ranked.get('expected_outcome_hopeful', '')
    }

    # Execute actions
    #response = hlp.execute_actions(action_request_payload, 2)  # The 2 is the wait_time parameter

    response = hlp.send_action_request(payload=action_request_payload, wait_time=2)

    actions_succeeded = False
    page_changed = False
    goal_achieved = False
    overall_goal_success = False


    if response:

        for at in page_action_json_ranked['action_tasks']:
            #print(at)
            for c in at['candidates']:
                #print(c)
                for r in response:
                    if r['action_id'] == at['action_id'] and r['candidate_id'] == c['candidate_id']:
                        # dusan_tracker.append("         -Acted on Action tasks: aID:" + str(r['action_id']) + " cID:" + str(r['candidate_id']))
                        if c.get('action','') != 'keyboard_action' and len(c.get("keyboard_action", '')) == 0:
                            c['element_metadata'] = r['element_metadata']
                        else:
                            c['element_metadata'] = {}
                        
        hlp.create_highlighted_screenshot_cairo(
            run_id=run_id,
            run_rerun_path=run_rerun_path, # Use the current run path since we take a screenshot after action,
            page_action_json_ranked=page_action_json_ranked,
            resize_screenshot_result=resize_screenshot_result
        )
        
        time.sleep(3)
            # Take screenshot AFTER the actions have been done, and use temp folder.
        url_metadata_temp, screenshot_data_temp = hlp.run_metadata_gather(
            output_path=run_rerun_path+"/temp",
            overlap_percentage=overlap_percentage,
            max_chunks=max_chunks
        )

        resize_screenshot_result_all_chunks_temp = {}
        temp_file_list =  hlp.get_files_with_extension(run_rerun_path+"/temp", ".png")
        for tf in temp_file_list:
            tmp_resize_screenshot_result = hlp.resize_and_crop(
                input_path=f'{run_rerun_path}/temp/{tf}',
                output_path=f'{run_rerun_path}/temp/resized_{tf}',
            )
            resize_screenshot_result_all_chunks_temp[tf] = tmp_resize_screenshot_result

        resize_screenshot_result_temp = resize_screenshot_result_all_chunks_temp['chunk_1.png']



        previous_step_detail = hlp.extract_entities_for_evaluation(page_action_json_ranked)

        eval_json = hlp.evaluate_run_screenshots(
            workflow_memory,
            site_wide_instructions,
            run_rerun_path,
            [resize_screenshot_result, resize_screenshot_result_temp],
            previous_step_detail,
            workflow_memory['summary'],
            start_time
        )
        logger.append_to_open_log_run_retry("Evaluation completed")

        actions_succeeded = eval_json['actions_succeeded']
        page_changed = eval_json['page_changed']
        overall_goal_success = eval_json['overall_goal_success']

        logger.set_run_retry_value("actions_succeeded", eval_json['actions_succeeded'])
        logger.set_run_retry_value("page_changed", eval_json['page_changed'])
        logger.set_run_retry_value("overall_goal_success", eval_json['overall_goal_success'])

        logger.set_run_retry_value("summary", eval_json['summary'])
        logger.set_run_retry_value("run_advice", eval_json['run_advice'])
        logger.set_run_retry_value("_expectations_analysis", eval_json['_expectations_analysis'])
        logger.set_run_retry_value("_precision_analysis", eval_json['_precision_analysis'])
        logger.set_run_retry_value("_error_type", eval_json['_error_type'])

        for a in eval_json['action_tasks']:
            logger.set_action(a['action_id'])
            logger.set_action_value(key="_error_type", value=c.get('_error_type', ''))

            for c in a.get('candidates', []):
                logger.set_candidate(c['candidate_id'])
                logger.set_candidate_value(key="success", value=c.get('success', ''))
                logger.set_candidate_value(key="issues", value=c.get('issues', ''))
                logger.set_candidate_value(key="_candidate_advice", value=c.get('_candidate_advice', ''))


        workflow_memory['summary'] = eval_json['summary']

        if not overall_goal_success:
            # dusan_tracker.append("    -didn't succeed in goal")
            advice, workflow_memory, summary = hlp.handle_negative_evaluation(
                eval_json,
                workflow_memory,
                page_action_json_ranked
            )

    else:
        #  dusan_tracker.append("    -No response from the action API")
            print("Error!!!!!!!!!!!!!!!No response from the action API")

    # track_data["runs"].append(run_data)
    # hlp.write_track_data(track_data, folder_structures['workflow_path'])

    if overall_goal_success:
        # dusan_tracker.append("    -Goal achieved and can continue")
        workflow_memory['run_db'][run_id] = {
            'evaluation': eval_json,
            'actions': page_action_json_ranked
        }
    


    # Update run_id based on success of actions
    # If any actions succeeded and can't contune, increment run_id
    if not actions_succeeded and page_changed:  # If we cannot continue (major error)
        print("blue",1)

        logger.append_to_open_log_run_retry("Run needs to reset. Incremeenting rerun count.")

        rerun_ct += 1
        run_id = 0
        run_rerun_ct = 0
        logger.set_rerun(rerun_ct)
        logger.set_run(run_id)
        logger.set_run_retry(run_rerun_ct)

        any_actions_succeeded = False
        workflow_memory = hlp.reset_workflow(
            workflow_id=workflow_id,
            workflow_instructions=workflow_instructions,
            extra_instructions=extra_instructions
        )

        # dusan_tracker.append("    -Cannot continue, resetting memory, rerun count: " + str(rerun_ct) + "navigation to initial state url metadata")


        hlp.navigate_to_url_from_metadata(initial_state_url_metadata['url_metadata'])
        workflow_memory['run_db'] = {}  # Reset run data (correct)

        # Do NOT increment run_id here!
    elif actions_succeeded and page_changed:  # If we can continue (minor or no changes)
        
        print("blue",2)
        logger.append_to_open_log_run_retry("Action succeeded, page changed.")
        # dusan_tracker.append(f"    -Can continue actions_succeeded: {actions_succeeded}, page_changed: {page_changed}")
        if not overall_goal_success:  
            
            print("blue",3)
            logger.append_to_open_log_run_retry("Goal not yet achieved")
            # dusan_tracker.append(f"    -Goal not achieved but progress made, goal_achieved: {goal_achieved}, actions_succeeded: {actions_succeeded}, page_changed: {page_changed}")
            run_id += 1  # Move to the next run (correct)
            logger.set_run(run_id)
            run_rerun_ct = 0
            logger.set_run_retry(run_rerun_ct)
        else:
            
            print("blue",4)
            logger.append_to_open_log_run_retry("Goal achieved")
    else: # If we can continue but the page hasn't changed
        
        print("blue",5)
        logger.append_to_open_log_run_retry("No progress made, but can continue at current step.")
        #dont change the run_id but increment the run rerun count
        # dusan_tracker.append(f"    -Page hasn't changed, goal_achieved: {goal_achieved}, actions_succeeded: {actions_succeeded}, page_changed: {page_changed}")
        run_rerun_ct += 1
        logger.set_run_retry(run_rerun_ct)
    

    #todo
    # why do I still get Error in main loop: Key 'summary_of_steps_so_far' already exists in this node; cannot overwrite.

{'workflow_id': 'QB_batch', 'sample_id': 3, 'rerun_id': 0, 'run_id': 0, 'run_retry_id': 0, 'action_id': None, 'candidate_id': None}
Now previous =  which is run 
And Run is now: c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\QB_batch\3\0\0\0
blue 2
blue 3
{'workflow_id': 'QB_batch', 'sample_id': 3, 'rerun_id': 0, 'run_id': 1, 'run_retry_id': 0, 'action_id': None, 'candidate_id': None}
Now previous = c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\QB_batch\3\0\0\0 which is run c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\QB_batch\3\0\0\0
And Run is now: c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\QB_batch\3\0\1\0
Copying screenshot from temp to new run folder: c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\QB_batch\3\0\0\0/temp to c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\QB_batch\3\0\1\0/chunks
blue 5
{'workflow_id': 'QB_batch', 'sample_id': 3, 'rerun_id': 0, 'run_id': 1, 'run_retry_id': 1, 'action_id': None, 'candidate_id': None}
Now previous = c:\Users\dusa

KeyboardInterrupt: 

In [5]:
workflow_memory

{'workflow_id': 'qb_contract',
 'workflow_instructions': 'Add a new contractor (one not already added) through the payroll / contractors page, clicking the add a contractor button and submitting the form.',
 'previous_state_advice': '',
 'extra_instructions': '',
 'run_db': {3: {'evaluation': {'_expectations_analysis': "The actions taken align with the expected workflow of adding a new contractor. The form fields were populated and the submit button was clicked. The outcome shows we've reached the contractor details page for the newly created Test Contractor LLC, indicating successful form submission.",
    'page_changed': True,
    'action_tasks': [{'action_id': 0,
      'candidates': [{'candidate_id': 0,
        'success': True,
        'issues': 'None',
        '_candidate_advice': 'The name field was correctly targeted and populated. The coordinates (510, 464) hit the input field accurately.'}],
      '_action_advice': 'The name field was appropriately selected and populated. No ch

In [42]:
overall_goal_success

False

In [33]:
print(f"Copying screenshot from temp to new run folder: {previous_run_rerun_path}/temp to {run_rerun_path}/chunks")
temp_file_list =  hlp.get_files_with_extension(previous_run_rerun_path+"/temp", ".png")
for tf in temp_file_list:
    print(f'{previous_run_rerun_path}/temp/{tf}',f'{run_rerun_path}/chunks/{tf}')

    hlp.copy_and_rename_file(
        f'{previous_run_rerun_path}/temp/{tf}',
        f'{run_rerun_path}/chunks/{tf}',
        delete_original=False
    )

Copying screenshot from temp to new run folder: c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\qb_pay_contractors5\4\0\0\0/temp to c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\qb_pay_contractors5\4\0\1\0/chunks
c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\qb_pay_contractors5\4\0\0\0/temp/chunk_1.png c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\qb_pay_contractors5\4\0\1\0/chunks/chunk_1.png
c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\qb_pay_contractors5\4\0\0\0/temp/chunk_2.png c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\qb_pay_contractors5\4\0\1\0/chunks/chunk_2.png
c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\qb_pay_contractors5\4\0\0\0/temp/chunk_3.png c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\qb_pay_contractors5\4\0\1\0/chunks/chunk_3.png
c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\qb_pay_contractors5\4\0\0\0/temp/resized_chunk_1.png c:\Users\dusan\OneDrive\Desktop\pp\test\crawl\v12\qb_pay_contractors5\4\0\1\0/chunks/resized_chunk_1.p

In [27]:
previous_run_rerun_path,run_rerun_path

('c:\\Users\\dusan\\OneDrive\\Desktop\\pp\\test\\crawl\\v12\\qb_pay_contractors5\\4\\0\\1\\0',
 'c:\\Users\\dusan\\OneDrive\\Desktop\\pp\\test\\crawl\\v12\\qb_pay_contractors5\\4\\0\\1\\0')

In [29]:
previous_run_rerun_path='c:\\Users\\dusan\\OneDrive\\Desktop\\pp\\test\\crawl\\v12\\qb_pay_contractors5\\4\\0\\0\\0'
run_rerun_path = 'c:\\Users\\dusan\\OneDrive\\Desktop\\pp\\test\\crawl\\v12\\qb_pay_contractors5\\4\\0\\1\\0'

In [35]:
logger.data['workflows']['qb_add_product_inventory']['samples'][4]

{'created_at': datetime.datetime(2024, 12, 29, 21, 58, 54, 881874),
 'open_log': [],
 'reruns': {0: {'created_at': datetime.datetime(2024, 12, 29, 21, 58, 54, 885872),
   'open_log': [],
   'runs': {0: {'created_at': datetime.datetime(2024, 12, 29, 21, 58, 54, 889870),
     'open_log': [],
     'run_retries': {0: {'created_at': datetime.datetime(2024, 12, 29, 21, 58, 54, 893867),
       'open_log': ['Evaluation completed',
        'No progress made, but can continue at current step.'],
       'summary_of_steps_so_far': 'No steps have been taken yet. This is the initial state.',
       'expected_outcome_hopeful': 'The system should open a new form or page specifically designed for creating a new product or service item in the inventory.',
       '_advice_assessment': 'No previous advice to assess as this is the first step.',
       'actions': {0: {'created_at': datetime.datetime(2024, 12, 29, 21, 59, 10, 550237),
         'open_log': [],
         'description': 'Click on the Product or 

In [46]:
workflow_name = 'c:\\Users\\dusan\\OneDrive\\Desktop\\pp\\test\\crawl\\v12\\amazon_gift\\0/1\\dots'
overlap_percentage=20
max_chunks=1

In [50]:
import requests

screenshot_response = requests.get(
    f"http://127.0.0.1:8000/screenshot",
    params={"workflow_step_path": workflow_name, "overlap_percentage": overlap_percentage,"max_chunks": max_chunks, 
                "action_id": None,
                "candidate_id": None
            }
)

Token refreshed successfully.
Found 1 messages from the last 30 minutes.
Email ID: 1945848dbb8db220
From: "dbosmail@gmail.com" <dbosmail@gmail.com>
Subject: Test D
Snippet: D Test
Body:
D Test




NameError: name 'service' is not defined

In [23]:
import requests

# Example: watch for the Intuit clickstream, wait 5s inactivity, but max 10s overall
payload = {
    "watch_url": "https://eventbus.intuit.com/v2/intuit-clickstream",
    "silence_ms": 3000,
    "max_runtime_ms": 20000
}

r = requests.post("http://127.0.0.1:8000/listen_for_requests", json=payload)
print("Status Code:", r.status_code)
print("JSON Response:", r.json())


Status Code: 200
JSON Response: {'captured_requests': [{'method': 'POST', 'request_url': 'https://eventbus.intuit.com/v2/intuit-clickstream?last_iac_request_rt=47', 'page_url': 'https://sandbox.qbo.intuit.com/app/cashflowoverview', 'post_data': '{"data_version":"w1.0","cec_version":"1","client_framework_version":"<CLIENT_LIBRARY_FRAMEWORK_VERSION>","application":{"app_name":"QuickBooksOnline2.0","forwarded_by_app_id":"","offering_id":"QuickBooksOnline","os":"Win32","platform":"web","browser":{"browser_version":"5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36","browser_name":"Netscape","locale":"en-US","browser_height":1144,"browser_width":1236,"screen_width":3440,"screen_height":1440,"screen_orientation":null,"screen_color_depth":24,"cookies_enabled":true,"encoding":"UTF-8","user_agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36","url":"","url_host_name":"sandbo

In [37]:
r.json()['captured_requests'][0]['post_data']

rr = hlp.parse_json_response(r.json()['captured_requests'][0]['post_data'])
rr

{'data_version': 'w1.0',
 'cec_version': '1',
 'client_framework_version': '<CLIENT_LIBRARY_FRAMEWORK_VERSION>',
 'application': {'app_name': 'QuickBooksOnline2.0',
  'forwarded_by_app_id': '',
  'offering_id': 'QuickBooksOnline',
  'os': 'Win32',
  'platform': 'web',
  'browser': {'browser_version': '5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
   'browser_name': 'Netscape',
   'locale': 'en-US',
   'browser_height': 1144,
   'browser_width': 1236,
   'screen_width': 3440,
   'screen_height': 1440,
   'screen_orientation': None,
   'screen_color_depth': 24,
   'cookies_enabled': True,
   'encoding': 'UTF-8',
   'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
   'url': '',
   'url_host_name': 'sandbox.qbo.intuit.com',
   'query_param': None,
   'browser_plugins': 'PDF Viewer;Chrome PDF Viewer;Chromium PDF Viewer;Microsoft Edge PDF Viewer;WebKit b

In [36]:
r.json()['captured_requests'][0]

{'method': 'POST',
 'request_url': 'https://eventbus.intuit.com/v2/intuit-clickstream?last_iac_request_rt=47',
 'page_url': 'https://sandbox.qbo.intuit.com/app/cashflowoverview',
 'post_data': '{"data_version":"w1.0","cec_version":"1","client_framework_version":"<CLIENT_LIBRARY_FRAMEWORK_VERSION>","application":{"app_name":"QuickBooksOnline2.0","forwarded_by_app_id":"","offering_id":"QuickBooksOnline","os":"Win32","platform":"web","browser":{"browser_version":"5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36","browser_name":"Netscape","locale":"en-US","browser_height":1144,"browser_width":1236,"screen_width":3440,"screen_height":1440,"screen_orientation":null,"screen_color_depth":24,"cookies_enabled":true,"encoding":"UTF-8","user_agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36","url":"","url_host_name":"sandbox.qbo.intuit.com","query_param":null,"browser_plugin

In [10]:
data

{'captured_requests': []}

In [4]:
logger.data['workflows']['qb_contnew']['samples']

{'0': {'created_at': datetime.datetime(2024, 12, 28, 21, 16, 4, 496224),
  'open_log': [],
  'reruns': {'0': {'created_at': datetime.datetime(2024, 12, 28, 21, 16, 4, 498223),
    'open_log': [],
    'runs': {'0': {'created_at': datetime.datetime(2024, 12, 28, 21, 16, 4, 501221),
      'open_log': [],
      'run_retries': {'0': {'created_at': datetime.datetime(2024, 12, 28, 21, 16, 4, 504219),
        'open_log': []}}}}}}},
 1: {'created_at': datetime.datetime(2024, 12, 28, 21, 39, 45, 544578),
  'open_log': [],
  'reruns': {0: {'created_at': datetime.datetime(2024, 12, 28, 21, 39, 45, 547576),
    'open_log': [],
    'runs': {0: {'created_at': datetime.datetime(2024, 12, 28, 21, 39, 45, 549575),
      'open_log': [],
      'run_retries': {0: {'created_at': datetime.datetime(2024, 12, 28, 21, 39, 45, 553573),
        'open_log': ['Evaluation completed',
         'No progress made, but can continue at current step.'],
        'summary_of_steps_so_far': 'No steps have been taken yet. Sta

In [ ]:


create_highlighted_screenshot(
        run_id=run_id,
        run_data=run_data,
        workflow_step_path_temp=workflow_step_path, # Use the current run path since we take a screenshot after action,
        page_action_json_ranked=page_action_json_ranked,
        resize_screenshot_result=resize_screenshot_result,
        highlighted_images_dir=highlighted_images_dir
)


Starting highlight creation...
Resize screenshot result: {'message': 'Image resized successfully', 'original_width': 1252, 'original_height': 1292, 'new_width': 1058, 'new_height': 1092}

Processing action task: Click the Add to Cart button
Found bounding box: {'x': 965.90625, 'y': 883, 'width': 127.09375, 'height': 32}

Drawing 1 bounding boxes

Debug - Drawing inputs:
Image path: addItem\0/chunk_1.png
Output path: addItem\highlighted_images\run_1_step_4_highlighted.png
Bounding boxes: [{'x': 965.90625, 'y': 883, 'width': 127.09375, 'height': 32}]
Successfully saved Cairo visualization to: addItem\highlighted_images\run_1_step_4_highlighted.png
Saved highlighted screenshot: addItem\highlighted_images\run_1_step_4_highlighted.png


In [62]:
response[0]['element_metadata']['boundingBox']

{'x': 1479.815058684857,
 'y': 1260.388066704816,
 'width': 338.05405595936264,
 'height': 48.03055707547919}

In [2]:
import get_2fa as g2
emails = g2.fetch_emails(minutes=3)
sms = g2.fetch_sms(limit = 5)





Token refreshed successfully.
Found 1 messages from the last 30 minutes.


In [14]:

sm = sms[1]
sm.from_,sm.to,sm.body



('+18583369123', '+18583303274', 'Dtest')

In [15]:
hlp.draw_elements(
    page_action_json_ranked,
    draw_dots=draw_dots,
    take_screenshot=False,
    workflow_step_path=workflow_step_path,
    diameter=20,
    max_chunks=max_chunks,
    resize_screenshot_result=resize_screenshot_result  # Add this
)


Starting draw_elements...
Input page_action_json: {
  "multiple_steps_required": false,
  "visible_elements_from_instructions": "The 'Add to Cart' button is clearly visible on the right side of the product page for the Titleist Perma-Soft Men's Golf Glove.",
  "summary_of_steps_so_far": "No previous actions taken.",
  "action_tasks": [
    {
      "description": "Click the Add to Cart button",
      "action_id": 0,
      "candidates": [
        {
          "element_description": "Yellow 'Add to Cart' button located on the right side of the product page, below the price of $21.98",
          "action": "click",
          "type_text": "",
          "expected_outcome": "Item should be added to cart and a confirmation message should appear",
          "keyboard_action": "",
          "coordinates": {
            "x": 840,
            "y": 651
          },
          "combined_score": 0.0,
          "coordinates_ready_to_act": {
            "x": 840,
            "y": 651
          },
       

([],
 {'multiple_steps_required': False,
  'visible_elements_from_instructions': "The 'Add to Cart' button is clearly visible on the right side of the product page for the Titleist Perma-Soft Men's Golf Glove.",
  'summary_of_steps_so_far': 'No previous actions taken.',
  'action_tasks': [{'description': 'Click the Add to Cart button',
    'action_id': 0,
    'candidates': [{'element_description': "Yellow 'Add to Cart' button located on the right side of the product page, below the price of $21.98",
      'action': 'click',
      'type_text': '',
      'expected_outcome': 'Item should be added to cart and a confirmation message should appear',
      'keyboard_action': '',
      'coordinates': {'x': 840, 'y': 651},
      'combined_score': 0.0,
      'coordinates_ready_to_act': {'x': 840, 'y': 651},
      'rank': 1,
      'to_act': True,
      'candidate_id': 0}]}],
  'error': '',
  'page_description': "This is an Amazon product page for a Titleist Perma-Soft Men's Golf Glove priced at $